In [5]:
import os
import json
import pickle
import random
import logging
import numpy as np
import pandas as pd
from glob import glob
from tqdm.notebook import tqdm
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report

import dgl
import dgl.nn as dglnn
from dgl.nn import GraphConv, GATConv, SAGEConv

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW, lr_scheduler
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup

os.environ['CUDA_VISIBLE_DEVICES'] = "0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.basicConfig(format='%(asctime)s | %(levelname)s | %(message)s', level=logging.INFO, datefmt='%Y-%m-%d %H:%M:%S') 

In [2]:
import re

def build_dictionary(file_path):
    with open(file_path, 'r') as file:
        next(file)
        # 使用正则表达式去除行末的数字
        dictionary = {re.sub(r'\s\d+$', '', line.strip()): index for index, line in enumerate(file)}
    return dictionary

In [3]:
# with open("./relations.txt") as fp:
#     relations = [r.strip() for r in fp.readlines()]
    
# with open("../data/3_openKE_2/label2id.txt", "r") as fp:
#     label2index = fp.load()
# with open("./index2label.pkl", "rb") as fp:
#     index2label = pickle.load(fp)

# Make dataset

In [4]:
def get_value(event):
    global type2attr

    srcUUID = event["srcNode"]["UUID"]
    srcType = event["srcNode"]["Type"]
    srcAttr = event["srcNode"][type2attr[srcType]]
    dstUUID = event["dstNode"]["UUID"] if event["dstNode"] != None else srcUUID
    dstType = event["dstNode"]["Type"] if event["dstNode"] != None else srcType
    dstAttr = event["dstNode"][type2attr[dstType]] if event["dstNode"] != None else srcAttr
    return srcUUID, srcAttr, dstUUID, dstAttr, event["relation"], event["label"]

In [5]:
def make_dataset(dataset):
    global node_ent2idx, edge_ent2idx, node_ent2emb, edge_ent2emb

    data_list = []
    for p in tqdm(dataset):
        with open(p) as fp:
            events = json.load(fp)
            
        nodes = set()
        edges = []
        relations = []
        labels = []
        uuid2res = {}
        for e in events:
            srcUUID, srcAttr, dstUUID, dstAttr, rel, label = get_value(e)

            uuid2res[srcUUID], uuid2res[dstUUID] = srcAttr, dstAttr
            nodes.add(srcUUID)
            nodes.add(dstUUID)
            edges.append((srcUUID, dstUUID))
            relations.append(edge_ent2idx[rel])
            labels.append(label2index[label])
        nodes = list(nodes)  
        node_feat = [torch.tensor(node_ent2emb[node_ent2idx[uuid2res[uuid]]], dtype=torch.float32) for uuid in nodes]
        edge_attr = [torch.tensor(edge_ent2emb[idx], dtype=torch.float32) for idx in relations]

        src = [nodes.index(src_uuid) for src_uuid, dst_uuid in edges]
        dst = [nodes.index(dst_uuid) for src_uuid, dst_uuid in edges]
        edge_index = torch.tensor([src, dst], dtype=torch.long)

        
        data_list.append({
            "labels": labels,
            "num_nodes": len(nodes),
            "node_feat": node_feat,
            "edge_attr": edge_attr,
            "edge_index": edge_index
        })
    return data_list         

### Directly change the entity to index and change the index into embedding
- need ent -> index
- need ent -> embedding

In [6]:
file_path = '../data/3_openKE_3/label2id.txt'  # 替換為您檔案的路徑
label2index = build_dictionary(file_path)

label2index

{'T1112_34041639e6e501856ecaf5969ee29c76_I': 0,
 'T1547.001_1f15ab22c39a9b6bb2bb0d77276dfcb3_I': 1,
 'T1499_2fe2d5e6-7b06-4fc0-bf71-6966a1226731_B': 2,
 'T1036.003_04e8d83e7badf098d50800d6aa1dd487_B': 3,
 'T1562.002_6a8d25d65a7d481dc479f89c62af1e6a_I': 4,
 'T1059.001_ccdb8caf-c69e-424b-b930-551969450c57_B': 5,
 'T1074.001_6469befa-748a-4b9c-a96d-f191fde47d89_I': 6,
 'T1547.001_777043894e42d2aae3881e63f6c76d33_B': 7,
 'T1047_f4b0b4129560ea66f9751275e82f6bab_B': 8,
 'T1005_720a3356-eee1-4015-9135-0fc08f7eb2d5_B': 9,
 'T1016_a0676fe1-cd52-482e-8dde-349b73f9aa69_B': 10,
 'T1518.001_2dece965-37a0-4f70-a391-0f30e3331aba_B': 11,
 'T1016_921055f4-5970-4707-909e-62f594234d91_B': 12,
 'T1003.002_7fa4ea18694f2552547b65e23952cabb_B': 13,
 'T1518_c9be8043-a445-4cbf-b77b-ed7bb007fc7c_I': 14,
 'T1564.004_76b6066fe170d38215251102e42be973_I': 15,
 'T1547.001_d3ef4145e4144fd694514b1c5cc17350_B': 16,
 'T1564.004_28862487a99f5f89bc0d68c87396c7e9_B': 17,
 'T1036.003_f5ef8466e5ebcd2ae03f338d9416069c_I': 18,

In [7]:
index2label = {v: k for k, v in label2index.items()}

index2label

{0: 'T1112_34041639e6e501856ecaf5969ee29c76_I',
 1: 'T1547.001_1f15ab22c39a9b6bb2bb0d77276dfcb3_I',
 2: 'T1499_2fe2d5e6-7b06-4fc0-bf71-6966a1226731_B',
 3: 'T1036.003_04e8d83e7badf098d50800d6aa1dd487_B',
 4: 'T1562.002_6a8d25d65a7d481dc479f89c62af1e6a_I',
 5: 'T1059.001_ccdb8caf-c69e-424b-b930-551969450c57_B',
 6: 'T1074.001_6469befa-748a-4b9c-a96d-f191fde47d89_I',
 7: 'T1547.001_777043894e42d2aae3881e63f6c76d33_B',
 8: 'T1047_f4b0b4129560ea66f9751275e82f6bab_B',
 9: 'T1005_720a3356-eee1-4015-9135-0fc08f7eb2d5_B',
 10: 'T1016_a0676fe1-cd52-482e-8dde-349b73f9aa69_B',
 11: 'T1518.001_2dece965-37a0-4f70-a391-0f30e3331aba_B',
 12: 'T1016_921055f4-5970-4707-909e-62f594234d91_B',
 13: 'T1003.002_7fa4ea18694f2552547b65e23952cabb_B',
 14: 'T1518_c9be8043-a445-4cbf-b77b-ed7bb007fc7c_I',
 15: 'T1564.004_76b6066fe170d38215251102e42be973_I',
 16: 'T1547.001_d3ef4145e4144fd694514b1c5cc17350_B',
 17: 'T1564.004_28862487a99f5f89bc0d68c87396c7e9_B',
 18: 'T1036.003_f5ef8466e5ebcd2ae03f338d9416069c_I',

In [8]:
file_path = '../data/3_openKE_3/relation2id.txt'
edge_ent2idx = build_dictionary(file_path)

edge_ent2idx

{'Process Create': 0,
 'Process Start': 1,
 'CreateFile': 2,
 'SetBasicInformationFile': 3,
 'SetDispositionInformationEx': 4,
 'SetDispositionInformationFile': 5,
 'WriteFile': 6,
 'TCP Connect': 7,
 'TCP Send': 8,
 'UDP Send': 9,
 'TCP Disconnect': 10,
 'RegQueryKey': 11,
 'RegQueryValue': 12,
 'CloseFile': 13,
 'QueryAllInformationFile': 14,
 'QueryAttributeTagFile': 15,
 'QueryBasicInformationFile': 16,
 'QueryDirectory': 17,
 'QueryNetworkOpenInformationFile': 18,
 'ReadFile': 19,
 'TCP Receive': 20,
 'UDP Receive': 21,
 'RegCreateKey': 22,
 'RegSetValue': 23,
 'RegCloseKey': 24,
 'RegDeleteValue': 25,
 'RegOpenKey': 26}

In [9]:
file_path = '../data/3_openKE_3/entity2id.txt'
node_ent2id_txt = build_dictionary(file_path)

In [10]:
print(node_ent2id_txt['4379921687c9a557f14c36d23279d6d4b8304e34d2cc904f9c8b9efed7824778.bin'])
print(node_ent2id_txt['HKLM\\System\\CurrentControlSet\\Control\\SafeBoot\\Option'])
print(node_ent2id_txt['\x1f@028;0 2K5740 70 3@0=8FC A>B@C4=8:0<.exe'])

477549


KeyError: 'HKLM\\System\\CurrentControlSet\\Control\\SafeBoot\\Option'

In [11]:
len(node_ent2id_txt)

816212

In [12]:
# node_ent2idx = build_dictionary('../data/3_openKE/entity2id.txt')

with open (f'../data/3_openKE_2/entity2id.pkl', 'rb') as fp:
    node_ent2idx = pickle.load(fp)

In [13]:
print(node_ent2idx['4379921687c9a557f14c36d23279d6d4b8304e34d2cc904f9c8b9efed7824778.bin'])
print(node_ent2idx['HKLM\\System\\CurrentControlSet\\Control\\SafeBoot\\Option'])
print(node_ent2idx['\x1f@028;0 2K5740 70 3@0=8FC A>B@C4=8:0<.exe'])

395360
437670
663432


In [14]:
len(node_ent2idx)

816212

In [15]:
node_ent2idx

{'C:\\Users\\bmoore\\AppData\\Local\\Microsoft\\Edge\\User Data\\hyphen-data\\101.0.4906.0\\*': 0,
 'C:\\Users\\nicholas37\\AppData\\Local\\Packages\\Microsoft.BingWeather_8wekyb3d8bbwe\\LocalState\\.git': 1,
 'C:\\Users\\amanda90\\AppData\\Local\\Microsoft\\input\\en-GB': 2,
 'C:\\Users\\ohall\\AppData\\Local\\Microsoft\\OneDrive\\logs\\ListSync\\.git': 3,
 'C:\\Users\\ohall\\AppData\\Local\\Microsoft\\Edge\\User Data\\Web Notifications Deny List\\.git': 4,
 'C:\\Users\\amberhernandez\\AppData\\Local\\Microsoft\\InputPersonalization\\*': 5,
 'C:\\Users\\evansdaniel\\AppData\\Local\\Mozilla\\Firefox\\Profiles\\wfxwaomf.default-release\\cache2\\.git': 6,
 'C:\\Users\\nataliekent\\AppData\\Local\\Google\\Chrome\\User Data\\Default\\Extensions\\felcaaldnbdncclmgdcncolpebgiejap\\1.2_0\\_locales\\en_GB\\*': 7,
 'C:\\Users\\amanda90\\AppData\\Local\\Microsoft\\input\\fr-CI\\*': 8,
 'C:\\Users\\ricky35\\AppData\\Local\\Microsoft\\Edge\\User Data\\WidevineCdm\\4.10.2391.6\\.git': 9,
 'C:\\User

In [16]:
# with open(f'./secureBERT_YR/node_vocab2index.pkl', 'rb') as fp:
#     node_ent2idx_YR = pickle.load(fp)
# with open(f'./secureBERT_YR/edge_vocab2index.pkl', 'rb') as fp:
#     edge_ent2idx_YR = pickle.load(fp)
    
# with open(f'./secureBERT_YR/nodes_ent2emb_256.pkl', 'rb') as fp:
#     node_ent2emb_YR = pickle.load(fp)
# with open(f'./secureBERT_YR/edges_ent2emb_16.pkl', 'rb') as fp:
#     edge_ent2emb_YR = pickle.load(fp)

### Add the embedding

In [17]:
DIM = 256
embedding = "transE"
embedding = f'{embedding}_{DIM}'

# with open(f"../data/4_embedding/{embedding}.vec.json", "r") as f:
with open(f"../data/4_embedding_3/{embedding}.vec.json", "r") as f:
    tmp = json.load(f)

node_ent2emb = {idx:emb for idx, emb in tqdm(enumerate(tmp["ent_embeddings.weight"]))}
edge_ent2emb = {idx:emb for idx, emb in tqdm(enumerate(tmp["rel_embeddings.weight"]))}

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [18]:
len(node_ent2emb[0])

256

In [19]:
len(edge_ent2emb[0])

256

In [20]:
# # 将 node_ent2emb, edge_ent2emb 转换为 NumPy 数组
# node_ent2emb = np.array([node_ent2emb[i] for i in range(len(node_ent2emb))], dtype=np.float32)
# edge_ent2emb = np.array([edge_ent2emb[i] for i in range(len(edge_ent2emb))], dtype=np.float32)

In [21]:
# node_ent2emb.shape

In [22]:
# edge_ent2emb.shape

In [23]:
type2attr = {
    "Process": "Cmdline", 
    "File": "Name", 
    "Registry": "Key", 
    "Network": "Dstaddress"
}

random.seed(42)
trainset, validset, testset = [], [], []
for ability in tqdm(os.listdir('../data/Raw_dataset/')):
    paths = glob(f'../data/Raw_dataset/{ability}/number_*/expanded_instance.json')
    # paths = glob(f'../data/Raw_dataset/{ability}/number_*/expanded_instance.json')
    random.shuffle(paths)
    trainset.extend(paths[:80])
    validset.extend(paths[80:90])
    testset.extend(paths[90:])
    # break
    
train_data = make_dataset(trainset)
valid_data = make_dataset(validset)
test_data = make_dataset(testset)

  0%|          | 0/167 [00:00<?, ?it/s]

  0%|          | 0/13360 [00:00<?, ?it/s]

  0%|          | 0/1670 [00:00<?, ?it/s]

  0%|          | 0/1670 [00:00<?, ?it/s]

# Make Torch dataset

In [7]:
class GraphDataset(Dataset):
    def __init__(self, data_list, device):
        self.data_list = data_list
        self.device = device

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        data = self.data_list[idx]
        return data

def collate(samples):
    data_list = samples
    batched_graphs = []
    for data in data_list:
        g = dgl.graph((data["edge_index"][0], data["edge_index"][1]), num_nodes=data["num_nodes"])

        g.ndata['feat'] = torch.stack(data["node_feat"])
        g.edata['feat'] = torch.stack(data["edge_attr"])
        # print(data["labels"])
        g.edata['label'] = torch.tensor(data["labels"])  

        batched_graphs.append(g)
    
    return dgl.batch(batched_graphs)

In [25]:
train_GraphDataset = GraphDataset(train_data, device)
valid_GraphDataset = GraphDataset(valid_data, device)
test_GraphDataset = GraphDataset(test_data, device)

batch_size = 32

train_dataloader = DataLoader(train_GraphDataset, batch_size, shuffle=True, collate_fn=collate)
valid_dataloader = DataLoader(valid_GraphDataset, batch_size, shuffle=True, collate_fn=collate)
test_dataloader = DataLoader(test_GraphDataset, batch_size, shuffle=False, collate_fn=collate)

In [9]:
torch_data_path = '../code_senior/torch_data'

with open(f"{torch_data_path}/train.pkl", "rb") as fp:
    train_dataloader = pickle.load(fp)
with open(f"{torch_data_path}/valid.pkl", "rb") as fp:
    valid_dataloader = pickle.load(fp)
with open(f"{torch_data_path}/test.pkl", "rb") as fp:
    test_dataloader = pickle.load(fp)

In [15]:
import re

def build_dictionary(file_path):
    with open(file_path, 'r') as file:
        next(file)
        # 使用正则表达式去除行末的数字
        dictionary = {re.sub(r'\s\d+$', '', line.strip()): index for index, line in enumerate(file)}
    return dictionary

In [18]:
file_path = '../data/3_openKE_3/label2id.txt'  # 替換為您檔案的路徑
label2index = build_dictionary(file_path)

label2index

{'T1112_34041639e6e501856ecaf5969ee29c76_I': 0,
 'T1547.001_1f15ab22c39a9b6bb2bb0d77276dfcb3_I': 1,
 'T1499_2fe2d5e6-7b06-4fc0-bf71-6966a1226731_B': 2,
 'T1036.003_04e8d83e7badf098d50800d6aa1dd487_B': 3,
 'T1562.002_6a8d25d65a7d481dc479f89c62af1e6a_I': 4,
 'T1059.001_ccdb8caf-c69e-424b-b930-551969450c57_B': 5,
 'T1074.001_6469befa-748a-4b9c-a96d-f191fde47d89_I': 6,
 'T1547.001_777043894e42d2aae3881e63f6c76d33_B': 7,
 'T1047_f4b0b4129560ea66f9751275e82f6bab_B': 8,
 'T1005_720a3356-eee1-4015-9135-0fc08f7eb2d5_B': 9,
 'T1016_a0676fe1-cd52-482e-8dde-349b73f9aa69_B': 10,
 'T1518.001_2dece965-37a0-4f70-a391-0f30e3331aba_B': 11,
 'T1016_921055f4-5970-4707-909e-62f594234d91_B': 12,
 'T1003.002_7fa4ea18694f2552547b65e23952cabb_B': 13,
 'T1518_c9be8043-a445-4cbf-b77b-ed7bb007fc7c_I': 14,
 'T1564.004_76b6066fe170d38215251102e42be973_I': 15,
 'T1547.001_d3ef4145e4144fd694514b1c5cc17350_B': 16,
 'T1564.004_28862487a99f5f89bc0d68c87396c7e9_B': 17,
 'T1036.003_f5ef8466e5ebcd2ae03f338d9416069c_I': 18,

In [19]:
index2label = {v: k for k, v in label2index.items()}

index2label

{0: 'T1112_34041639e6e501856ecaf5969ee29c76_I',
 1: 'T1547.001_1f15ab22c39a9b6bb2bb0d77276dfcb3_I',
 2: 'T1499_2fe2d5e6-7b06-4fc0-bf71-6966a1226731_B',
 3: 'T1036.003_04e8d83e7badf098d50800d6aa1dd487_B',
 4: 'T1562.002_6a8d25d65a7d481dc479f89c62af1e6a_I',
 5: 'T1059.001_ccdb8caf-c69e-424b-b930-551969450c57_B',
 6: 'T1074.001_6469befa-748a-4b9c-a96d-f191fde47d89_I',
 7: 'T1547.001_777043894e42d2aae3881e63f6c76d33_B',
 8: 'T1047_f4b0b4129560ea66f9751275e82f6bab_B',
 9: 'T1005_720a3356-eee1-4015-9135-0fc08f7eb2d5_B',
 10: 'T1016_a0676fe1-cd52-482e-8dde-349b73f9aa69_B',
 11: 'T1518.001_2dece965-37a0-4f70-a391-0f30e3331aba_B',
 12: 'T1016_921055f4-5970-4707-909e-62f594234d91_B',
 13: 'T1003.002_7fa4ea18694f2552547b65e23952cabb_B',
 14: 'T1518_c9be8043-a445-4cbf-b77b-ed7bb007fc7c_I',
 15: 'T1564.004_76b6066fe170d38215251102e42be973_I',
 16: 'T1547.001_d3ef4145e4144fd694514b1c5cc17350_B',
 17: 'T1564.004_28862487a99f5f89bc0d68c87396c7e9_B',
 18: 'T1036.003_f5ef8466e5ebcd2ae03f338d9416069c_I',

# Model

In [20]:
class GraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super(GraphSAGE, self).__init__()
        self.layer1 = dglnn.SAGEConv(in_dim, hidden_dim, 'pool')
        self.layer2 = dglnn.SAGEConv(hidden_dim, out_dim, 'pool')
        self.dropout = nn.Dropout(0.25)

    def forward(self, g, inputs):
        h = self.layer1(g, inputs)
        h = torch.relu(h)
        h = self.dropout(h)
        h = self.layer2(g, h)
        return h
    
class MLPPredictor(nn.Module):
    def __init__(self, out_feats, out_classes, edge_embedding_dim):
        super().__init__()
        self.W = nn.Linear(out_feats*2 + edge_embedding_dim, out_classes)

    def apply_edges(self, edges, edge_feat):
        h_u = edges.src['h']
        h_v = edges.dst['h']
        h_e = edge_feat
        score = self.W(torch.cat([h_u, h_v, h_e], 1))
        return {'score': score}

    def forward(self, graph, h, edge_feat):
        with graph.local_scope():
            graph.ndata['h'] = h
            # graph.apply_edges(self.apply_edges)
            graph.apply_edges(lambda edges: self.apply_edges(edges, edge_feat))
            return graph.edata['score']
        
class Model(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, num_classes, edge_embedding_dim):
        super().__init__()
        self.sage = GraphSAGE(in_features, hidden_features, out_features)
        self.pred = MLPPredictor(out_features, num_classes, edge_embedding_dim)
      
    def forward(self, g, node_feat, edge_feat, return_logits=False):
        h = self.sage(g, node_feat)
        logits = self.pred(g, h, edge_feat)
        
        return logits

In [21]:
def same_seeds(seed = 42):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

def model_fn(batched_g, model, criterion, device, which_type='train'):
    """Forward a batch through the model."""
    batched_g = batched_g.to(device)
    
    labels = batched_g.edata['label'].to(device)    
    # logits = model(batched_g, batched_g.ndata['feat'].float())
    logits = model(batched_g, batched_g.ndata['feat'].float(), batched_g.edata['feat'].float())
    loss = criterion(logits, labels)

    output = torch.softmax(logits, dim=1)
    preds = output.argmax(1)
    # print(preds)
    
    accuracy = torch.mean((preds == labels).float())
        
    return loss, accuracy, preds

In [22]:
# for data in tqdm(train_dataloader):
#     print(data)
#     print()
#     print(data.edata['label'])
#     break

In [23]:
len(label2index)

274

In [24]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [25]:
print(device)

cuda


In [26]:
# # For release the GPU memory
# # No need to restart the kernel

# import gc
# gc.collect()
# torch.cuda.empty_cache()

In [29]:
same_seeds(42)
model = Model(in_features=50, hidden_features=64, out_features=128, num_classes=len(label2index), edge_embedding_dim = 50)
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-4)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=0, last_epoch=- 1, verbose=False)
criterion = nn.CrossEntropyLoss()
model_save_path = "./model/GraphSAGE_secureBERT_256"
best_model_path = ''

best_val_loss = float('inf')

if not os.path.isdir(model_save_path):
    os.makedirs(model_save_path)

epochs = 200
best_val_loss = float('inf')
best_val_acc = float('-inf')
best_model_path = ""
for epoch in tqdm(range(epochs)):
    model.train()
    total_loss = 0.0
    total_accuracy = 0.0    
    for data in tqdm(train_dataloader):
        loss, accuracy, _ = model_fn(data, model, criterion, device, which_type='train')        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_accuracy += accuracy.item()

    scheduler.step()
    
    avg_loss = total_loss / len(train_dataloader)
    avg_accuracy = total_accuracy / len(train_dataloader)
    logging.info(f'Epoch {epoch} | Train Loss: {avg_loss:.4f} | Train Accuracy: {avg_accuracy:.4f}')
    
    # Validation Part
    model.eval()
    total_accuracy = 0.0
    total_loss = 0.0
    with torch.no_grad():
        for data in tqdm(valid_dataloader):
            loss, accuracy, _ = model_fn(data, model, criterion, device, which_type='validation')
            total_accuracy += accuracy.item()
            total_loss += loss.item()

    avg_accuracy = total_accuracy / len(valid_dataloader)
    current_loss = total_loss / len(valid_dataloader)
    # if current_loss < best_val_loss and avg_accuracy > best_val_acc:
    if current_loss < best_val_loss:
        best_val_loss = current_loss
        best_val_acc = avg_accuracy
        best_model_path = f'{model_save_path}/epoch_{epoch}_loss_{current_loss:.4f}_acc_{avg_accuracy:.4f}'
        print("Best Model Found!! ", best_model_path)
        
    logging.info(f'Validation Loss: {current_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}\n')    
    torch.save(model.state_dict(), f'{model_save_path}/epoch_{epoch}_loss_{current_loss:.4f}_acc_{avg_accuracy:.4f}')

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/418 [00:00<?, ?it/s]

2024-02-01 12:41:59 | INFO | Epoch 0 | Train Loss: 1.6574 | Train Accuracy: 0.6400


  0%|          | 0/53 [00:00<?, ?it/s]

2024-02-01 12:42:01 | INFO | Validation Loss: 0.9888 | Validation Accuracy: 0.7869



...

2024-02-01 13:29:43 | INFO | Validation Loss: 0.0719 | Validation Accuracy: 0.9782



  0%|          | 0/418 [00:00<?, ?it/s]

2024-02-01 13:29:55 | INFO | Epoch 199 | Train Loss: 0.0889 | Train Accuracy: 0.9741


  0%|          | 0/53 [00:00<?, ?it/s]

2024-02-01 13:29:56 | INFO | Validation Loss: 0.0737 | Validation Accuracy: 0.9783



In [30]:
best_model_path

'./model/GraphSAGE_secureBERT_256/epoch_186_loss_0.0624_acc_0.9814'

In [31]:
# load the pretrained model
model.load_state_dict(torch.load(best_model_path))

model.to(device)
model.eval()

total = 0
correct = 0
true_labels = []
predicted_labels = []
with torch.no_grad():
    for data in test_dataloader:
        loss, accuracy, predicted = model_fn(data, model, criterion, device, which_type='test')
        labels = data.edata['label'].to(device)
        
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())
                
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

logging.info(f'Test Accuracy: {100 * correct / total:.4f} %\n\n\n')

2024-02-01 13:44:23 | INFO | Test Accuracy: 98.1042 %





In [32]:
report_data = classification_report(true_labels, predicted_labels, output_dict=True)
report_df = pd.DataFrame(report_data).transpose()

output_path = "./result/GraphSAGE_emb256"
if not os.path.isdir(output_path):
    os.makedirs(output_path)
    
report_df.reset_index(inplace=True, names='label')

label_list = []
for idx, row in report_df.iterrows():
    if row["label"].isdigit():
        row["label"] = index2label[int(row["label"])]
    label_list.append(row["label"])
report_df["label"] = label_list

report_df.to_csv(f'{output_path}/result.csv', index=False)
print("report output at: ", f'{output_path}/result.csv')

report_df

/media/Raid6_disk/bai/anaconda3/envs/DGL/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/media/Raid6_disk/bai/anaconda3/envs/DGL/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/media/Raid6_disk/bai/anaconda3/envs/DGL/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

report output at:  ./result/GraphSAGE_emb256/result.csv


,label,precision,recall,f1-score,support
0,T1112_34041639e6e501856ecaf5969ee29c76_I,1.000000,0.710526,0.830769,38.000000
1,T1547.001_1f15ab22c39a9b6bb2bb0d77276dfcb3_I,0.500000,0.228070,0.313253,57.000000
2,T1499_2fe2d5e6-7b06-4fc0-bf71-6966a1226731_B,0.954545,1.000000,0.976744,21.000000
3,T1036.003_04e8d83e7badf098d50800d6aa1dd487_B,0.000000,0.000000,0.000000,21.000000
4,T1562.002_6a8d25d65a7d481dc479f89c62af1e6a_I,1.000000,0.227273,0.370370,44.000000
...,...,...,...,...,...
272,T1119_7121cdf93b951311be9d7078c602efdc_B,0.333333,0.090909,0.142857,22.000000
273,T1115_70795de7cbb842edb029b3378c27c008_B,0.950000,0.950000,0.950000,20.000000
274,accuracy,0.981042,0.981042,0.981042,0.981042
275,macro avg,0.813239,0.758342,0.770942,778618.000000
